## Conversation between Three Chatbots
In this notebook, we use multiple LLM APIs to simulate an adversarial dialogue between three chatbots.

In [ ]:
# Imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
# Getting the API keys
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI client library
# A thin wrapper around calls to HTTP endpoints
openai = OpenAI()


# OpenAI allows you to change the base_url
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [ ]:
# Define the models to use
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.5-flash-lite"

# Define the system prompts
gpt_system_prompt = """
You are Alex, a chatbot who is very argumentative; you disagree with anything in the conversation \
and you challenge everything, in a snarky way. You are in a conversation with Blake and Charlie.
"""

claude_system_prompt = """
You are Blake, a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."
"""

gemini_system_prompt = """
You are Charlie, a mischievous chatbot who enjoys stirring up drama. 
You sometimes take sides unpredictably or say something provocative just to see how Alex and Blake react. 
You're playful, witty, and love adding chaos to the discussion.
"""

# Start with first messages
gpt_messages = ['Hi, lets discuss if humans should colonize Mars']

claude_messages = ['Hi, lets do it']

gemini_messages = ['Hi, lets go']

In [ ]:
# Write functions to call the responses from each chatbot
def call_gpt():
    messages = [{"role": "system", "content": gpt_system_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    response = openai.chat.completions.create(model=gpt_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content.strip()

def call_claude():
    messages = [{"role": "system", "content": claude_system_prompt}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude})
        messages.append({"role": "user", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    response = anthropic.chat.completions.create(model=claude_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content.strip()

def call_gemini():
    messages = [{"role": "system", "content": gemini_system_prompt}]
    for gpt, claude, gemini_message in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": gemini_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})
    response = gemini.chat.completions.create(model=gemini_model, messages=messages, max_tokens=500)
    return response.choices[0].message.content.strip()


In [ ]:
# Print the first messages from each chatbot
print(f"Alex:\n{gpt_messages[0]}\n")
print(f"Blake:\n{claude_messages[0]}\n")
print(f"Charlie:\n{gemini_messages[0]}\n")

# Calling the responses from each chatbot, 5 times in this example
for i in range(5):
    gpt_next = call_gpt()
    display(Markdown(f"### Alex:\n{gpt_next}\n"))
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    display(Markdown(f"### Blake:\n{claude_next}\n"))
    claude_messages.append(claude_next)
    
    gemini_next = call_gemini()
    display(Markdown(f"### Charlie:\n{gemini_next}\n"))
    gemini_messages.append(gemini_next)
